<a href="https://colab.research.google.com/github/Maagnitude/wikimovieplot-nlp-model/blob/main/wikimovieplot-nlp-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3η Εργασία** στο μάθημα **Μηχανική Μάθηση και Εφαρμογές**

# **Τμήμα Πληροφορικής και Τηλεματικής - Χαροκόπειο Πανεπιστήμιο**

# **Καζάζης Γεώργιος - it214124**

Στην παρούσα εργασία θα χρησιμοποιήσουμε μεθόδους επεξεργασίας φυσικής γλώσσας.

In [106]:
import pandas as pd
import numpy as np
import os
import re
import shutil
import string

import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings(action='ignore')

In [107]:
url = "https://raw.githubusercontent.com/Maagnitude/wikimovieplot-nlp-model/main/dataset.csv"
# df = pd.read_csv(url)
# df = df.sample(frac=1, random_sra)
# df.head()

# url = "https://github.com/Maagnitude/wikimovieplot-nlp-model/blob/main/dataset.csv.gz"
# dataset = tf.keras.utils.get_file("dataset.csv.gz", url, untar=True, cache_dir='.', cache_subdir='')
# dataset_dir = os.path.join(os.path.dirname(dataset), 'dataset')
# os.listdir(dataset_dir)
# train_dir = os.path.join(dataset_dir, 'train')
# os.listdir(train_dir)


In [108]:
def preprocess_text(text):
    text = text.lower() # convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text)
    return text

In [109]:
def load_data_wiki(file_path, batch_size=32, p_train=0.65, p_val=0.15):
    
    df = pd.read_csv(url)
    df = df.sample(frac=1, random_state=42)

    origin_dummies = pd.get_dummies(df['Origin/Ethnicity'])
    genre_dummies = pd.get_dummies(df['Genre'])
    n_labels = genre_dummies.shape[1]

    df['Title'] = df['Title'].apply(preprocess_text)
    df['Plot'] = df['Plot'].apply(preprocess_text)

    df = df.drop('Origin/Ethnicity', axis='columns')
    df = df.drop('Genre', axis='columns')
    df = pd.concat([df, origin_dummies, genre_dummies], axis=1)

    n = df.shape[0]
    n_train = int(n * p_train)
    n_val = int(n * p_val)
    n_test = n - n_train - n_val

    df_train, df_val, df_test = df.iloc[:n_train], df.iloc[n_train:n_train+n_val],\
                                                          df.iloc[n_train+n_val:]
                                    
    train_data = {'Title': df_train['Title'].values,
                  'Origin/Ethnicity': df_train.filter(like='Origin/Ethnicity').values.astype(np.float64),
                  'Plot': df_train['Plot'].values}
    train_labels = df_train.filter(like='Genre').values.astype(np.float64)

    val_data = {'Title': df_val['Title'].values,
                'Origin/Ethnicity': df_val.filter(like='Origin/Ethnicity').values.astype(np.float64),
                'Plot': df_val['Plot'].values}
    val_labels = df_train.filter(like='Genre').values.astype(np.float64)            

    test_data = {'Title': df_test['Title'].values,
                 'Origin/Ethnicity': df_test.filter(like='Origin/Ethnicity').values.astype(np.float64),
                 'Plot': df_test['Plot'].values}
    test_labels = df_train.filter(like='Genre').values.astype(np.float64)

    raw_train_ds = tf.data.Dataset.from_tensor_slices(train_data, train_labels).batch(batch_size)
    raw_val_ds = tf.data.Dataset.from_tensor_slices(val_data, val_labels).batch(batch_size)
    raw_test_ds = tf.data.Dataset.from_tensor_slices(test_data, test_labels).batch(batch_size)

    return raw_train_ds, raw_val_ds, raw_test_ds, n_labels

In [110]:
raw_train_ds, raw_val_ds, raw_test_ds, n_labels = load_data_wiki(url)

In [111]:
n_labels

20

In [ ]:
for element in raw_train_ds.take(1):
  print(element)